In [1]:
# Import libraries 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, date, timedelta
from pandas.tseries.offsets import MonthBegin, MonthEnd
import pyarrow
import dask
import dask.dataframe as dd
import utilities as nori
from utilities import *

# Connect to ODBC
import pyodbc
conn = pyodbc.connect(DSN='Radar',UID='ruser',PWD='8uzUTHub')

# Dates
currentDt = pd.datetime.now().date()
transDt = ((currentDt + pd.DateOffset(months=6)) - pd.Timedelta(pd.offsets.Day(376))).strftime('%m/%d/%Y 00:0:0')

date1 = (currentDt - (pd.Timedelta(pd.offsets.Day(365))) + MonthBegin(-2)).strftime('%m/%d/%Y 00:0:0')
date2 = (currentDt - (pd.Timedelta(pd.offsets.Day(366))) + MonthEnd(6)).strftime('%m/%d/%Y 00:0:0')

In [1]:
# Published Fare
query1 = """SELECT
  REVENUE.RecordLocator AS PNR,
  TRVL_CALENDAR.DateDate AS TrvlDt,
  BKG_CALENDAR.DateDate AS BkDt,
  AIRPORT_POO.AirportCode AS Org,
  AIRPORT_POD.AirportCode AS Dest,
  AIRPORT_POO.IsDomestic AS OrgDom,
  AIRPORT_POD.IsDomestic AS DestDom,
  (ROUTE_LEG.IndustryMiles) AS Miles,
  count( distinct REVENUE.PassengerSrcID ) AS Pax,
  Sum(REVENUE.RevenueAmount) AS Rev
FROM
  CalendarDim  BKG_CALENDAR INNER JOIN RevenueFact  REVENUE ON (REVENUE.BookingDateSID=BKG_CALENDAR.DateSID)
   INNER JOIN ClassOfServiceDim  CLASSOFSERVICE ON (REVENUE.ClassOfServiceSID=CLASSOFSERVICE.ClassOfServiceSID  AND  CLASSOFSERVICE.IsCurrent=1)
   INNER JOIN TransactionStatusDim  TRANS_STATUS ON (REVENUE.TransactionStatusSID=TRANS_STATUS.TransactionStatusSID)
   INNER JOIN TransactionTypeDim  TRANS_TYPE ON (TRANS_TYPE.TransactionTypeSID=REVENUE.TransactionTypeSID)
   INNER JOIN CalendarDim  TRVL_CALENDAR ON (REVENUE.DepartureDateSID=TRVL_CALENDAR.DateSID)
   INNER JOIN RouteDim  ROUTE_LEG ON (REVENUE.LegRouteSID=ROUTE_LEG.RouteSID  AND  ROUTE_LEG.IsCurrent=1)
   INNER JOIN AirportDim  AIRPORT_POO ON (AIRPORT_POO.AirportSID=REVENUE.POO_SID)
   INNER JOIN AirportDim  AIRPORT_POD ON (AIRPORT_POD.AirportSID=REVENUE.POD_SID)
   INNER JOIN ChannelDim  BKG_CHANNEL ON (BKG_CHANNEL.ChannelSID=REVENUE.BookingChannelSID)
  
WHERE
  (
   TRVL_CALENDAR.DateDate  BETWEEN  '%s'  AND  '%s'
   AND
   BKG_CALENDAR.DateDate <= '%s'
   AND
   AIRPORT_POO.AirportCode  <>  'UNK'
   AND
   AIRPORT_POD.AirportCode  <>  'UNK'
   AND
   TRANS_STATUS.StatusTier3  IN  ( 'Balance Due','Boarded (flown)','Booked','Expired','No Show','Unknown'  )
   AND
   TRANS_TYPE.TypeTier3  =  'Published Fare'
   AND
   CLASSOFSERVICE.bool_NonRev  =  0
  )
GROUP BY
  REVENUE.RecordLocator, 
  TRVL_CALENDAR.DateDate, 
  BKG_CALENDAR.DateDate, 
  AIRPORT_POO.AirportCode, 
  AIRPORT_POD.AirportCode, 
  AIRPORT_POO.IsDomestic, 
  AIRPORT_POD.IsDomestic, 
  (ROUTE_LEG.IndustryMiles)
"""%(date1,date2,transDt)
df1 = pd.read_sql(query1,conn,parse_dates=['TrvlDt','BkDt'])
df1 = df1.groupby(['PNR','TrvlDt','BkDt','Org','Dest','OrgDom','DestDom'],as_index=False).agg({'Pax':'max','Rev':'sum','Miles':'sum'})

# Baggage
query2 = """SELECT
  REVENUE.RecordLocator AS PNR,
  TRVL_CALENDAR.DateDate AS TrvlDt,
  TRANS_CALENDAR.DateDate AS TransDt,
  AIRPORT_POO.AirportCode AS Org,
  AIRPORT_POD.AirportCode AS Dest,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG1' THEN REVENUE.PassengerSrcID ELSE NULL END) as Bag1,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG1' THEN REVENUE.RevenueAmount ELSE 0 END) as Bag1Rev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG2' THEN REVENUE.PassengerSrcID ELSE NULL END) as Bag2,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG2' THEN REVENUE.RevenueAmount ELSE 0 END) as Bag2Rev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 IN ( 'Gate Carry-On','Carry-On' ) THEN REVENUE.PassengerSrcID ELSE NULL END) as CarryOn,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 IN ( 'Gate Carry-On','Carry-On' ) THEN REVENUE.RevenueAmount ELSE 0 END) as CarryOnRev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 IN ( 'Gate Carry-On', 'Carry-On','BAG1','BAG2','BAG3','BAG4','BAG5' ) THEN REVENUE.PassengerSrcID ELSE NULL END) as Cum,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 IN ( 'Gate Carry-On','Carry-On','BAG1','BAG2','BAG3','BAG4','BAG5' ) THEN REVENUE.RevenueAmount ELSE 0 END) as CumRev

FROM
  ChannelDim  TRANS_CHANNEL INNER JOIN RevenueFact  REVENUE ON (REVENUE.TransactionChannelSID=TRANS_CHANNEL.ChannelSID)
   INNER JOIN ClassOfServiceDim  CLASSOFSERVICE ON (REVENUE.ClassOfServiceSID=CLASSOFSERVICE.ClassOfServiceSID  AND  CLASSOFSERVICE.IsCurrent=1)
   INNER JOIN TransactionStatusDim  TRANS_STATUS ON (REVENUE.TransactionStatusSID=TRANS_STATUS.TransactionStatusSID)
   INNER JOIN TransactionTypeDim  TRANS_TYPE ON (TRANS_TYPE.TransactionTypeSID=REVENUE.TransactionTypeSID)
   INNER JOIN CalendarDim  TRVL_CALENDAR ON (REVENUE.DepartureDateSID=TRVL_CALENDAR.DateSID)
   INNER JOIN CalendarDim  TRANS_CALENDAR ON (REVENUE.TransactionDateSID=TRANS_CALENDAR.DateSID)
   INNER JOIN RouteDim  ROUTE_LEG ON (REVENUE.LegRouteSID=ROUTE_LEG.RouteSID  AND  ROUTE_LEG.IsCurrent=1)
   INNER JOIN AirportDim  AIRPORT_POO ON (AIRPORT_POO.AirportSID=REVENUE.POO_SID)
   INNER JOIN AirportDim  AIRPORT_POD ON (AIRPORT_POD.AirportSID=REVENUE.POD_SID)
  
WHERE
  (
   TRVL_CALENDAR.DateDate  BETWEEN  '%s'  AND  '%s'
   AND
   TRANS_CALENDAR.DateDate <= '%s'
   AND
   AIRPORT_POO.AirportCode  <>  'UNK'
   AND
   AIRPORT_POD.AirportCode  <>  'UNK'
   AND
   TRANS_STATUS.StatusTier3  IN  ( 'Balance Due','Boarded (flown)','Booked','Expired','No Show','Unknown'  )
   AND
   TRANS_TYPE.TypeTier3  IN  ( 'BAG1','BAG2','BAG3','BAG4','BAG5','Carry-On','Gate Carry-On'  )
   AND
   rtrim(REVENUE.FeeCode)  NOT IN  ( 'BMLC','BML1','BML2','BAGE','BAGN','BAGP'  )
   AND
   CLASSOFSERVICE.bool_NonRev  =  0
  )
GROUP BY
  REVENUE.RecordLocator, 
  TRVL_CALENDAR.DateDate,
  TRANS_CALENDAR.DateDate,
  AIRPORT_POO.AirportCode, 
  AIRPORT_POD.AirportCode 
HAVING
  Sum(REVENUE.RevenueAmount)  >=  0
"""%(date1,date2,transDt)
df2 = pd.read_sql(query2,conn,parse_dates=['TrvlDt','TransDt'])

df = pd.merge(df1,df2,how='left',on=['PNR','TrvlDt','Org','Dest']) # Merge and convert to Dask
df = dd.from_pandas(df,npartitions=3)

# Data Wrangling
df = nori.month(df) # Month Extraction
df = nori.diff(df) # DTD

df = nori.journey(df) # Domestic
df['Domestic'] = df.Domestic.apply(domestic, meta='bool')

trip = nori.duration(df) # Trip Duration
df = df.merge(trip,how='left',on='PNR') 

df['Miles_Group'] = df.Miles.apply(miles, meta='object') # Feature binning
df['DTD_Group'] = df.DTD.apply(dtd, meta='object')
df['Pax_Group'] = df.Pax.apply(pax, meta='object')

df = nori.cluster(df)

# Groupby
df = df.groupby(
        ['Cluster','Miles_Group','DTD_Group','Pax_Group','TrvlMo','Domestic']
    ).agg({'Pax':'sum','Rev':'mean','Duration':'mean',
           'CarryOn':'sum','CarryOnRev':'sum','Bag1':'sum',
           'Bag1Rev':'sum','Bag2':'sum','Bag2Rev':'sum',
           'Cum':'sum','CumRev':'sum'}).reset_index()
# Metrics
df = nori.bag1(df)
df = nori.bag2(df)
df = nori.carryon(df)
df = nori.cum(df)

df.to_parquet('lyf {}.parquet'.format((currentDt + MonthBegin(-1)).strftime('%Y-%m-%d'))) # Save as parquet file

In [2]:
df = pd.read_parquet('lyf {}.parquet'.format((currentDt + MonthBegin(-1)).strftime('%Y-%m-%d')))
df.to_csv('BagClusterLYF.csv')